In [4]:
import numpy as np
import xml.etree.ElementTree as ET
import os
import pandas as pd
import json
import scipy.stats
from utils import create_til_score
from multiprocessing import Pool
from tqdm.auto import tqdm

In [6]:
ground_truth = pd.read_csv(
    "/home/u1910100/Documents/Tiger_Data/wsitils/tiger-til-scores-wsitils.csv"
)
# print(ground_truth['image-id'], ground_truth['tils-score'])

til_score_dict = {}

for i, row in ground_truth.iterrows():
    id = row["image-id"]
    score = row["tils-score"]
    til_score_dict[id] = score

print(til_score_dict)

with open("ground_truth_tils.json", "w") as file:
    json.dump(til_score_dict, file)

{'104S': 70, '105S': 10, '106S': 10, '107S': 5, '108S': 30, '110S': 1, '112B': 1, '121S': 5, '123S': 60, '126B': 10, '129S': 40, '132B': 1, '139B': 40, '146B': 1, '155B': 3, '159S': 1, '160B': 75, '161S': 20, '162S': 50, '165B': 1, '174S': 85, '175B': 20, '184B': 5, '205B': 1, '208B': 1, '211B': 20, '212B': 5, '214B': 1, '223B': 1, '224B': 40, '227S': 95, '228B': 20, '233B': 1, '236B': 70, '244B': 1, '247B': 10, '248B': 10, 'TC_S01_P000004_C0001_B103': 95, 'TC_S01_P000006_C0001_B206': 90, 'TC_S01_P000020_C0001_B207': 5, 'TC_S01_P000041_C0001_B101': 5, 'TC_S01_P000045_C0001_B106': 20, 'TC_S01_P000047_C0001_B101': 1, 'TC_S01_P000049_C0001_B101': 3, 'TC_S01_P000052_C0001_B101': 95, 'TC_S01_P000055_C0001_B101': 65, 'TC_S01_P000058_C0001_B101': 1, 'TC_S01_P000061_C0001_B101': 65, 'TC_S01_P000063_C0001_B101': 5, 'TC_S01_P000064_C0001_B101': 1, 'TC_S01_P000073_C0001_B101': 30, 'TC_S01_P000074_C0001_B101': 5, 'TC_S01_P000077_C0001_B101': 10, 'TC_S01_P000079_C0001_B101': 15, 'TC_S01_P000083_C00

In [5]:
wsi_name = "104S.tif"
wsi_without_ext = os.path.splitext(wsi_name)[0]
cell_points_dir = "/home/u1910100/GitHub/TIAger-Torch/output/det_out"
cell_points_path = os.path.join(cell_points_dir, f"{wsi_without_ext}_points.json")
wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images/"
wsi_path = os.path.join(wsi_dir, wsi_name)
xml_path = "/home/u1910100/GitHub/TIAger-Torch/xml/105S.xml"
bulk_path = "/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/105S_bulk.npy"
create_til_score(wsi_path, xml_path, bulk_path)

|2024-04-30|14:59:39.397| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|14:59:39.398| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).


26

In [7]:
xml_dir = "/home/u1910100/GitHub/TIAger-Torch/xml"
pred_til_score_dict = {}
wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images/"


for xml_file in tqdm(os.listdir(xml_dir)):
    xml_path = os.path.join(xml_dir, xml_file)
    slide_name = os.path.splitext(xml_file)[0]

    bulk_path = (
        f"/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/{slide_name}_bulk.npy"
    )
    if not os.path.exists(bulk_path):
        bulk_path = f"/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/{slide_name}_seg.npy"
    if not os.path.exists(bulk_path):
        continue
    wsi_path = os.path.join(wsi_dir, f"{slide_name}.tif")
    score = create_til_score(wsi_path, xml_path, bulk_path)

    pred_til_score_dict[slide_name] = score

print(pred_til_score_dict)

  0%|          | 0/82 [00:00<?, ?it/s]

|2024-04-30|14:59:47.681| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|14:59:47.682| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).
|2024-04-30|14:59:47.990| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|14:59:47.990| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).
|2024-04-30|14:59:48.871| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|14:59:48.872| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).
|2024-04-30|14:59:59.402| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|14:59:59.403| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).
|2024-04-30|15:00:04.636| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-04-30|15:00:04.636| [WARNING] Metad

{'123S': 100, '214B': 10, '162S': 100, '212B': 100, '159S': 11, 'TC_S01_P000123_C0001_B101': 100, 'TC_S01_P000119_C0001_B101': 100, 'TC_S01_P000127_C0001_B101': 41, 'TC_S01_P000089_C0001_B101': 64, 'TC_S01_P000077_C0001_B101': 38, 'TC_S01_P000160_C0001_B204': 100, 'TC_S01_P000041_C0001_B101': 43, 'TC_S01_P000020_C0001_B207': 39, '211B': 64, 'TC_S01_P000073_C0001_B101': 95, 'TC_S01_P000116_C0001_B101': 100, 'TC_S01_P000120_C0001_B101': 73, '227S': 100, 'TC_S01_P000085_C0001_B101': 51, 'TC_S01_P000155_C0001_B101': 28, '205B': 2, 'TC_S01_P000106_C0001_B101': 71, 'TC_S01_P000112_C0001_B101': 100, '174S': 100, '110S': 4, '108S': 29, 'TC_S01_P000181_C0001_B101': 9, 'TC_S01_P000064_C0001_B101': 38, 'TC_S01_P000128_C0001_B101': 34, 'TC_S01_P000052_C0001_B101': 100, 'TC_S01_P000055_C0001_B101': 100, '160B': 100, '233B': 33, 'TC_S01_P000058_C0001_B101': 21, '161S': 46, 'TC_S01_P000083_C0001_B101': 23, '236B': 45, '224B': 100, 'TC_S01_P000139_C0001_B101': 54, 'TC_S01_P000108_C0001_B101': 100, '15

In [8]:
print(len(pred_til_score_dict))

pred_tils = []
true_tils = []
for slide_id in pred_til_score_dict.keys():
    pred_tils.append(pred_til_score_dict[slide_id])
    true_tils.append(til_score_dict[slide_id])

print(scipy.stats.pearsonr(true_tils, pred_tils))

81
PearsonRResult(statistic=0.6968197882402061, pvalue=5.022453629306933e-13)
